# Image Scaling with Machine Learning
## Inputs:
    1) Number of image pairs
    2) dictionary of {size_label_A: [pixels_high, pixels_wide], 
                      size_label_B: [pixels_high, pixels_wide]}
    3) run directory
    4) output directory
    5) existing sets parameters.json (None for first run)
***
## Parameters to generate a set of images for train-test-validate
    * Over all parameters
        * number of image sets
        * sizes dict {name1: [h, w], name2: [h, w],...}
        * run-limits {escape_boundry: diagonal_multiplier, max_iterations: int}
        * coloring scheme - perhaps a function
        * output file format
        * ouput file naming format
    * File system
        * output directory {train: path, test: path, validate: path}
        * run-log location
    * Stocastic Choices:
        * hashed dictionary - to reject duplicate images
        * equation
            * parameters
        * rotation 
        * center point 
        * frame scale

## Source structure
    * main - opens run_parameters file - passes to module function
    * selection and tracking module
    * starting plain complex
    * equations module
    * iteration module
    * matrix to image module
***
[PIL image](https://pillow.readthedocs.io/en/5.1.x/reference/Image.html#the-image-class) <br>
****
[GitHub scalygraphic](https://github.com/dlanier/scalygraphic/) <br>
### Background Code:
[Add: random plane on domain](https://github.com/dlanier/scalygraphic/blob/master/scalygraphic/zplain.py) <br>

#### Import & Expand *functions_demo.py*
[FMF functions_demo](https://github.com/dlanier/FlyingMachineFractal/blob/master/pyreimpic/functions_demo_01.py) <br>
#### Import & Trim itergators.py
[FMF itergators](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/itergataters.py) <br>
#### Rewrite HSV coloring
[FMF graphic_utility.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/graphic_utility.py) <br>
[FMF numcolorpy.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/numcolorpy.py) <br>
#### Reslove plain functions
[FMF z_plane.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/z_plane.py) <br>
****
    
### Inception: Random Selection, Tracking and Generation - called from *main()*

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Clickety Clack:
## production_parameters: *output_dir_name, image_sizes_dict, number_of_sets*
### get a random eq & p with a random domain
#### check uniquness
### get a random color map
## Iterate: run, write, report

In [45]:
#    ../src/im_scale_products.py
import os
import sys
import time
import hashlib
import inspect
from tempfile import TemporaryDirectory

import numpy as np
import PIL

sys.path.insert(0, '../src/')

import zplain as zp
import eq_iter
import deg_0_ddeq
import numcolorpy as ncp

"""     Constants (lookups):

        EQUS_DICT is an enumerated dictionary of the functions in module deg_0_ddeq
        EQUS_DICT_NAMED_IDX is a dictionary - name: number index of EQUS_DICT 
"""
EQUS_DICT = {k: v for k, v in enumerate(inspect.getmembers(deg_0_ddeq, inspect.isfunction))}
EQUS_DICT_NAMED_IDX = {v[0]: k for k, v in EQUS_DICT.items()}

def get_rand_eq_p_set():
    """ get a random equation and parameter set from the deg_0_ddeq module
    (No Args:)
    Returns:
        tuple:      (function_name, function_handle, parameter_set)
    """
    n = np.random.randint(0,len(EQUS_DICT),1)
    fcn_name, fcn = EQUS_DICT[n[0]]
    p = fcn(0.0, None)

    return (fcn_name, fcn, p)

def get_eq_by_name(fcn_name):
    """ Usage: function_handle = get_eq_by_name(fcn_name)
    Args:
        name of a function in deg_0_ddeq
    """
    if fcn_name in EQUS_DICT_NAMED_IDX:
        return EQUS_DICT[EQUS_DICT_NAMED_IDX[fcn_name]][1]
    else:
        return None
    
def get_random_domain(bounds_dict=None):
    """ Usage: 
    domain_dict = get_random_domain(h, w, bounds_dict)
    
    Args:
        bounds_dict:    min - max limits for keys eg.
                            CP_magnitude_limits = {'min': 0, 'max': 7}
                            ZM_limits           = {'min': np.finfo(float).eps, 'max': 2}
                            theta_limits        = {'min': 0, 'max':2 * np.pi}
                        
    Returns:
        domain_dict:    with keys:
                            center_point
                            zoom
                            theta
    """
    domain_dict = {}
    if bounds_dict is None:
        CP_magnitude_limits =   {'min': 0, 'max': 2}
        ZM_limits =             {'min': np.finfo(float).eps, 'max': 1}
        theta_limits =          {'min': 0, 'max':2 * np.pi}
    else:
        CP_magnitude_limits =   bounds_dict['CP_magnitude_limits']
        ZM_limits =             bounds_dict['ZM_limits']
        theta_limits =          bounds_dict['theta_limits']

    r = np.random.uniform(low=0.0, high=2*np.pi) * 0.0+1.0j
    m = np.random.uniform(low=CP_magnitude_limits['min'], high=CP_magnitude_limits['max'])
    domain_dict['center_point'] = m*np.exp(r)
    domain_dict['zoom'] = np.random.uniform(low=ZM_limits['min'], high=ZM_limits['max'])
    domain_dict['theta'] = np.random.uniform(low=theta_limits['min'], high=theta_limits['max'])
    
    return domain_dict

def sha256sum(s):
    """ Usage:  hash_key = sha256sum(input_str)
    """
    h  = hashlib.sha256()
    h.update(bytes(s, 'ascii'))
    
    return h.hexdigest()

def hash_parameters(domain_dict, fcn_name, p):
    """ Usage: hash_key = hash_parameters(domain_dict, fcn_name, p)
    """
    N_DEC = 15
    f = zp.get_frame_from_dict(domain_dict)
    s = zp.complex_frame_dict_to_string(f, N_DEC) + '\n' + fcn_name
    if isinstance(p, list):
        p_str = ''
        for p_n in p:
            p_str += zp.complex_to_string(p_n, N_DEC)
    else:
        p_str = zp.complex_to_string(p, N_DEC)
    
    s += p_str
    
    return sha256sum(s)

def get_im(ET, Z, Z0, domain_dict):
    """ Usage: I = get_im(ET, Z, Z0)
    """
    Zd, Zr, ETn = ncp.etg_norm(Z0, Z, ET)

    A = np.zeros((domain_dict['n_rows'], domain_dict['n_cols'],3))
    A[:,:,0] += ETn     # Hue
    A[:,:,1] += Zr      # Saturation
    A[:,:,2] += Zd      # Value
    I = PIL.Image.fromarray(np.uint8(A * 255), 'HSV').convert('RGB')
    
    return I

def get_gray_im(ET, Z, Z0, et_gray=False):
    """ Usage: I = get_im(ET, Z, Z0)
    """
    Zd, Zr, ETn = ncp.etg_norm(Z0, Z, ET)
    
    if et_gray:
        I = PIL.Image.fromarray(np.uint8(ETn * 255), 'L')
    else:
        I = PIL.Image.fromarray(np.uint8(Zd * 255), 'L')
    
    return I

def now_name(prefi_str=None, suffi_str=None):
    """ get a human readable time stamp name """
    t0 = time.time()
    t_dec = t0 - np.floor(t0)
    ahora_nombre = time.strftime("%a_%d_%b_%Y_%H_%M_%S", time.localtime()) 
    if prefi_str is None: prefi_str = ''
    if suffi_str is None: suffi_str = ''
        
    return prefi_str + '_' + ahora_nombre + suffi_str


def write_n_image_sets(n_2_do, iteration_dict, small_scale, large_scale, output_directory, hash_list=[]):
    """Usage:
                hash_list = write_n_image_sets( n_2_do, 
                                                iteration_dict, 
                                                small_scale, 
                                                large_scale, 
                                                output_directory, 
                                                hash_list )
    """
    if os.path.isdir(output_directory) == False:
        os.makedirs(output_directory)
                
    print(now_name('Write %i image pairs to \n%s\nStart '%(n_2_do, output_directory)))
    
    if len(hash_list) > 0:
        print('checking duplicates using input hash_list size = %i'%(len(hash_list)))
    else:
        print('new hash list started')

    with TemporaryDirectory() as test_temporary_dir:

        for k_do in range(n_2_do):
            fcn_name, eq, p = get_rand_eq_p_set()
            domain_dict = get_random_domain()

            domain_dict['it_max'] = iteration_dict['it_max']
            domain_dict['max_d'] = iteration_dict['scale_dist'] / domain_dict['zoom']

            hash_idx = hash_parameters(domain_dict, fcn_name, p)
            if hash_idx in hash_list:
                pass
            else:
                hash_list.append(hash_idx)
                domain_dict['n_rows'] = small_scale[0]
                domain_dict['n_cols'] = small_scale[1]
                domain_dict['dir_path'] = test_temporary_dir

                list_tuple = [(eq, (p))]

                t0 = time.time()
                ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
                I = get_im(ET, Z, Z0, domain_dict)
                file_name = os.path.join(output_directory, hash_idx + '_' + 'small.jpg')
                I.save(file_name)

                domain_dict['n_rows'] = large_scale[0]
                domain_dict['n_cols'] = large_scale[1]

                ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
                I = get_im(ET, Z, Z0, domain_dict)
                file_name = os.path.join(output_directory, hash_idx + '_' + 'large.jpg')
                I.save(file_name)
                print('\n%3i of %3i) %s\t\t'%(k_do+1, n_2_do, fcn_name), 
                      '%0.3f seconds (large & small image written)\n'%(time.time() - t0), 
                      hash_idx)
                
    print('\n', now_name('%i pairs written,\nFinished '%(k_do + 1)))
    
    return hash_list


# Demos:

****
## get a random function and parameter set

In [46]:
n_trys = 5

for k in range(n_trys):
    fcn_name, eq, p = get_rand_eq_p_set()
    
    print('\n\t', fcn_name)
    try:
        Z = 0.0+0.0j
        print(Z)
        print(eq(Z,p))
        Z = 1.0+1.0j
        print(Z)
        print(eq(Z,p))
    except:
        print('Crash crash ')
        break
        pass


	 RoyalZ
0j
inf
(1+1j)
(0.3279228087512498-0.8529740622329691j)

	 Nautuliz
0j
inf
(1+1j)
(0.7890985453389898-0.18370730130703805j)

	 ElGato
0j
inf
(1+1j)
(1.3861649239416862-0.16582757079924598j)

	 de_Okeeffe
0j
(-6.019728155339806+0.057856310679611646j)
(1+1j)
(-0.2221851582698301+0.2551022026406169j)

	 IslaLace
0j
inf
(1+1j)
(1.4559758142625245-1.102089495118849j)


## get a random complex domain

In [47]:
N_DEC=4

domain_dict = get_random_domain()
f = zp.get_frame_from_dict(domain_dict)
s = zp.complex_frame_dict_to_string(f, N_DEC)
print(s,'\n%50s'%('(from dictionary)'))
for k, v in domain_dict.items():
    if isinstance(v, complex):
        v_str = zp.complex_to_string(v)
    else:
        v_str = '%i'%(v)
        
    print('%40s: %s'%(k, v_str))

upper_left:1.7416-3.2670j	top_center:-0.9721-1.5438j	upper_right:-3.6857+0.1794j
left_center:3.4648-0.5534j	center_point:1.5023+2.3397j	right_center:-1.9625+2.8931j
bottom_left:5.1881+2.1603j	bottom_center:2.4744+3.8835j	bottom_right:-0.2393+5.6068j
 
                                 (from dictionary)
                            center_point: 0.751+1.170j
                                    zoom: 0
                                   theta: 2


## *hash_parameters( )* - function, parameters & domain - (reject duplicate parameters)
    Option: hashed dict > json file to allow reproduction of an exact data-set with different scaling

In [48]:
fcn_name, eq, p = get_rand_eq_p_set()
domain_dict = get_random_domain()

hash_key = hash_parameters(domain_dict, fcn_name, p)
print(hash_key)

ce0914821e2eb728f59a486da04d85cb4e7bd14461d64fea2736a6977395fbe6


## Produce many pair of scaled images

In [49]:
hash_list=[]

In [51]:
cell_start_time = time.time()

n_2_do = 2
iteration_dict = {'it_max': 64, 'scale_dist': 10}
small_scale = [128, 128]
large_scale = [255, 255]

output_directory = '../../ImagesSunday'
if os.path.isdir(output_directory) == False:
    os.makedirs(output_directory)

hash_list = write_n_image_sets(n_2_do, iteration_dict, small_scale, large_scale, output_directory, hash_list)

tt = time.time() - cell_start_time
print('\ntotal cell time: %0.2f seconds'%(tt))

Write 2 image pairs to 
../../ImagesSunday
Start _Sun_22_Sep_2019_16_35_40
checking duplicates using input hash_list size = 2

  1 of   2) RoyalZ		 12.046 seconds (large & small image written)
 4ec440f62d94fdd06d4dd22e6714de388a6c5c9f8130ecdb87f0b4b391d64c22

  2 of   2) RoyalZ		 20.156 seconds (large & small image written)
 7937e44ed77c804619683fea7a499314ebbbda61787cfab44d919dd37e5ac8d5

 2 pairs written,
Finished _Sun_22_Sep_2019_16_36_12

total cell time: 32.21 seconds


## file nameing ?

In [ ]:
do_da = now_name(prefi_str=fcn_name, suffi_str='.jpg')
print(do_da)
d0_dat = hashy_list[0] + '.jpg'
print(d0_dat)

## coloring logic
    * HSV model - choose three result components
        * Hue        (red to red) (0, 1) * 255          == visible spectrum colors
        * Saturation (gray to solid color) (0, 1) * 255 == color intensity
        * Value      (black to white) (0, 1) * 255      == brightness
    * RGB conversion from HSV
        * portable for multiple file formats
        * norm for train - validate - test
    * Escape-time algorithm produces
        * Escape Time (integer matrix)
        * Final Vector (Z - Z0)
            * distance (float matrix)
            * rotation (float matrix)
****
### Coloring component normalization:


In [ ]:
def view_component_color_range(Z0, Z, ET):
    # Examine color ranges - for HSV - RGB conversion 
    Zd_t, Zr_t, ETn_t = ncp.etg_norm(Z0, Z, ET)
    print('\nZd_t', np.max(Zd_t), np.min(Zd_t))
    print('Zr_t', np.max(Zr_t), np.min(Zr_t))
    print('ETn_t', np.max(ETn_t), np.min(ETn_t))